In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('Processed_Data.csv',lineterminator='\n')
df

,Unnamed: 0,topic,tweet,sentiment,class,processed_tweets
0,0,#olympics,Aussies would be happy that the T20 series hap...,0.275000,Positive,aussie would happy series happen midst olympic...
1,1,#olympics,The worst thing about the #Olympics finishing ...,-0.133333,Negative,worst thing olympics finish whole week availab...
2,2,#olympics,#Olympics\n\nWe play for India: #Hockey captai...,0.000000,Neutral,olympics play india hockey captain ranirampal ...
3,3,#olympics,See the best moments from the #Tokyo2020 closi...,1.000000,Positive,see best moment tokyo close ceremony videoelep...
4,4,#olympics,Fabulous! #Olympics \n#LoveTheBBC \n\nTokyo Ol...,0.500000,Positive,fabulous olympics lovethebbc tokyo olympics bb...
...,...,...,...,...,...,...
140242,140242,Tokyo olympics,Congratulations to all our winners and partici...,0.500000,Positive,congratulation winner participant olympics win...
140243,140243,Tokyo olympics,I hope that I am wrong but I have seen no twee...,-0.500000,Negative,hope wrong see tweet government
140244,140244,Tokyo olympics,Tokyo passes the baton to Paris as strangest e...,0.000000,Neutral,tokyo pass baton paris strangest ever olympic ...
140245,140245,Tokyo olympics,"Paris plans to deliver inclusive, youth-centre...",0.000000,Neutral,paris plan deliver inclusive youth centre gend...


In [27]:
def ratio(x):
  if x=='Positive':
    return 1
  elif x=='Negative':
    return 2
  else:
    return 0;

In [28]:
df['label'] = df['class'].apply(ratio)
df

,Unnamed: 0,topic,tweet,sentiment,class,processed_tweets,label
0,0,#olympics,Aussies would be happy that the T20 series hap...,0.275000,Positive,aussie would happy series happen midst olympic...,1
1,1,#olympics,The worst thing about the #Olympics finishing ...,-0.133333,Negative,worst thing olympics finish whole week availab...,2
2,2,#olympics,#Olympics\n\nWe play for India: #Hockey captai...,0.000000,Neutral,olympics play india hockey captain ranirampal ...,0
3,3,#olympics,See the best moments from the #Tokyo2020 closi...,1.000000,Positive,see best moment tokyo close ceremony videoelep...,1
4,4,#olympics,Fabulous! #Olympics \n#LoveTheBBC \n\nTokyo Ol...,0.500000,Positive,fabulous olympics lovethebbc tokyo olympics bb...,1
...,...,...,...,...,...,...,...
140242,140242,Tokyo olympics,Congratulations to all our winners and partici...,0.500000,Positive,congratulation winner participant olympics win...,1
140243,140243,Tokyo olympics,I hope that I am wrong but I have seen no twee...,-0.500000,Negative,hope wrong see tweet government,2
140244,140244,Tokyo olympics,Tokyo passes the baton to Paris as strangest e...,0.000000,Neutral,tokyo pass baton paris strangest ever olympic ...,0
140245,140245,Tokyo olympics,"Paris plans to deliver inclusive, youth-centre...",0.000000,Neutral,paris plan deliver inclusive youth centre gend...,0


In [29]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [30]:
df['processed_tweets'] = df['processed_tweets'].astype(str)

In [31]:

x = df['processed_tweets']
y = df['label']

In [32]:
num_words = 8000
embed_dim = 32
tokenizer = Tokenizer(num_words=num_words,oov_token = "<oov>" )
tokenizer.fit_on_texts(x)
word_index=tokenizer.word_index
sequences = tokenizer.texts_to_sequences(x)
length=[]
for i in sequences:
    length.append(len(i))
print(len(length))
print("Mean is: ",np.mean(length))
print("Max is: ",np.max(length))
print("Min is: ",np.min(length))

140247
Mean is:  9.267485222500303
Max is:  24
Min is:  1


In [33]:
pad_length = 24
sequences = pad_sequences(sequences, maxlen = pad_length, truncating = 'pre', padding = 'post')
sequences.shape

(140247, 24)

In [34]:
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import confusion_matrix, accuracy_score

In [36]:
x_train,x_test,y_train,y_test = tts(sequences,y,test_size = 0.05)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(133234, 24)
(7013, 24)
(133234,)
(7013,)


In [37]:
from keras.layers import Dense, Embedding, Dropout , Activation, Flatten, SimpleRNN
from keras.layers import GlobalMaxPool1D
from keras.models import Model, Sequential
import tensorflow as tf

In [38]:
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()

model = Sequential([Embedding(num_words, embed_dim, input_length = pad_length),
                   SimpleRNN(8, return_sequences = True),
                   GlobalMaxPool1D(),
                   Dense(20,activation = 'relu',kernel_initializer='he_uniform'),
                   Dropout(0.25),
                   Dense(3,activation = 'softmax')])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 24, 32)            256000    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 24, 8)             328       
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 8)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 20)                180       
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 63        
Total params: 256,571
Trainable params: 256,571
Non-trainable params: 0
________________________________________________

In [39]:
history = model.fit(x = x_train, y = y_train, epochs = 10,validation_split = 0.05)

Epoch 1/10
3956/3956 [==============================] - 39s 10ms/step - loss: 0.4874 - accuracy: 0.8237 - val_loss: 0.3134 - val_accuracy: 0.8996
Epoch 2/10
3956/3956 [==============================] - 38s 10ms/step - loss: 0.2973 - accuracy: 0.9077 - val_loss: 0.2994 - val_accuracy: 0.9045
Epoch 3/10
3956/3956 [==============================] - 37s 9ms/step - loss: 0.2582 - accuracy: 0.9200 - val_loss: 0.2887 - val_accuracy: 0.9099
Epoch 4/10
3956/3956 [==============================] - 38s 10ms/step - loss: 0.2328 - accuracy: 0.9286 - val_loss: 0.2937 - val_accuracy: 0.9095
Epoch 5/10
3956/3956 [==============================] - 38s 10ms/step - loss: 0.2138 - accuracy: 0.9341 - val_loss: 0.2919 - val_accuracy: 0.9113
Epoch 6/10
3956/3956 [==============================] - 38s 10ms/step - loss: 0.2003 - accuracy: 0.9386 - val_loss: 0.3008 - val_accuracy: 0.9087
Epoch 7/10
3956/3956 [==============================] - 38s 10ms/step - loss: 0.1878 - accuracy: 0.9421 - val_loss: 0.3160 - 

In [40]:
train_acc = history.history['accuracy']
valid_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

In [41]:
predictions = model.predict(x_test)
predict = []
for i in predictions:
    predict.append(np.argmax(i))

In [42]:
from sklearn import metrics

print(metrics.classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91      2880
           1       0.92      0.93      0.93      3409
           2       0.80      0.76      0.78       724

    accuracy                           0.91      7013
   macro avg       0.88      0.87      0.87      7013
weighted avg       0.91      0.91      0.91      7013

